Check if virtual environment is active

In [2]:
import sys
print(sys.prefix)
sys.prefix != sys.base_prefix # Should result in true

/home/addy/Desktop/Heart-Disease-Prediction-Capstone-Project-Group-55/.venv


True

Imports

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

2024-11-29 21:33:53.645368: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 21:33:54.110957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732896234.290808 1251438 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732896234.342919 1251438 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 21:33:54.794624: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Reading dataset

In [4]:
df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [5]:
df.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

Label Encoding (Feature Scaling)

In [6]:
cols_to_enc = ['ChestPainType', 'ExerciseAngina', 'RestingECG', 'Sex', 'ST_Slope']
enc = LabelEncoder()
for col in cols_to_enc:
    if col in df.columns:
        df[col] = enc.fit_transform(df[col])

df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0


Training neural network model

In [7]:
y = df['HeartDisease']
X = df.drop('HeartDisease', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = ModelCheckpoint('Models/Neural Network/neuralNetworkModel.keras', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

model.fit(X_train, y_train, epochs=2000, batch_size=64, validation_split=0.1, callbacks=[checkpoint], verbose=1)

best_model = tf.keras.models.load_model('Models/Neural Network/neuralNetworkModel.keras')


Epoch 1/2000


I0000 00:00:1732896238.068003 1251438 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3043 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
/home/addy/Desktop/Heart-Disease-Prediction-Capstone-Project-Group-55/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1732896239.180847 1252003 service.cc:148] XLA service 0x78b250006440 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732896239.181115 1252003 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-11-29 21:33:59.258631: I tensorflow/compil

 1/11 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.4219 - loss: 3.3914

I0000 00:00:1732896240.842750 1252003 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.4773 - loss: 2.5321
Epoch 1: val_accuracy improved from -inf to 0.59459, saving model to Models/Neural Network/neuralNetworkModel.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 263ms/step - accuracy: 0.4786 - loss: 2.4913 - val_accuracy: 0.5946 - val_loss: 0.6132
Epoch 2/2000
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4531 - loss: 1.5153
Epoch 2: val_accuracy did not improve from 0.59459
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5010 - loss: 1.2954 - val_accuracy: 0.5541 - val_loss: 0.6454
Epoch 3/2000
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4219 - loss: 1.1350
Epoch 3: val_accuracy did not improve from 0.59459
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5331 - loss: 0.9557 - val_accuracy: 0.5811 - val_loss: 0.6612
Epoch 4/2000
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5156 - loss: 0.9292
Epoch 4: val_accuracy improved from 0.59459 to 0.60811, saving model to Models/Neural Network/neura

Testing and Classification Report

In [8]:
probabilities = best_model.predict(X_test)

threshold = 0.5

binary_preds = np.where(probabilities.flatten() > threshold, 1, 0)

print("Classification Report: ")
print(classification_report(y_test, binary_preds))

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step
Classification Report: 
              precision    recall  f1-score   support

           0       0.72      0.88      0.80        77
           1       0.90      0.76      0.82       107

    accuracy                           0.81       184
   macro avg       0.81      0.82      0.81       184
weighted avg       0.83      0.81      0.81       184



Monte Carlo Dropout

In [9]:
class MonteCarloDropout(Model):
    def __init__(self, **kwargs):
        super(MonteCarloDropout, self).__init__(**kwargs)
        self.dense1 = Dense(128, activation='relu')
        self.dropout1 = Dropout(0.2)
        
        self.dense2 = Dense(64, activation='relu')
        self.dropout2 = Dropout(0.2)

        self.dense3 = Dense(32, activation='relu')
        self.dropout3 = Dropout(0.2)

        self.output_layer = Dense(1, activation='sigmoid')
    
    def call(self, inputs, training=None):
        x = self.dense1(inputs)
        x = self.dropout1(x, training=True) # keeping dropout active

        x = self.dense2(inputs)
        x = self.dropout2(x, training=True) # keeping dropout active

        x = self.dense3(inputs)
        x = self.dropout3(x, training=True) # keeping dropout active

        return self.output_layer(x)

mcModel = MonteCarloDropout()
mcModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

mcCheckpoint = ModelCheckpoint('mcDropoutModel.keras', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

mcModel.fit(X_train, y_train, epochs=2000, batch_size=64, validation_split=0.1, callbacks=[mcCheckpoint], verbose=1)

finalMCModel = tf.keras.models.load_model('mcDropoutModel.keras', custom_objects={'MonteCarloDropout': MonteCarloDropout})

def monteCarloPrediction(model, X, passes=100):
    predictions = np.stack([model(X, training=True).numpy().flatten() for _ in range(passes)], axis=0)
    return predictions

monteCarloPredictions = monteCarloPrediction(finalMCModel, X_test, passes=100)

mean = monteCarloPredictions.mean(axis=0)
std = monteCarloPredictions.std(axis=0)     # measure of uncertainity

mcThreshold = 0.5
mc_bin_pred = np.where(mean > mcThreshold, 1, 0)

Epoch 1/2000


/home/addy/Desktop/Heart-Disease-Prediction-Capstone-Project-Group-55/.venv/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:735: UserWarning: Gradients do not exist for variables ['monte_carlo_dropout/dense_6/kernel', 'monte_carlo_dropout/dense_6/bias', 'monte_carlo_dropout/dense_7/kernel', 'monte_carlo_dropout/dense_7/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4341 - loss: 26.5077
Epoch 1: val_accuracy improved from -inf to 0.58108, saving model to mcDropoutModel.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 113ms/step - accuracy: 0.4353 - loss: 26.2736 - val_accuracy: 0.5811 - val_loss: 10.9739
Epoch 2/2000
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4844 - loss: 21.2230
Epoch 2: val_accuracy did not improve from 0.58108
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4596 - loss: 17.0879 - val_accuracy: 0.5270 - val_loss: 13.0724
Epoch 3/2000
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5000 - loss: 16.0697
Epoch 3: val_accuracy did not improve from 0.58108
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4498 - loss: 15.3934 - val_accuracy: 0.4324 - val_loss: 13.3206
Epoch 4/2000
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4062 - loss: 16.6571
Epoch 4: val_accuracy did not improve from 0.58108
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4494 - loss:

In [10]:
print("Classification Report: ")
print(classification_report(y_test, mc_bin_pred))

print("Uncertainity Estimates (Standard Deviation of Predictions): ")
print(std)

Classification Report: 
              precision    recall  f1-score   support

           0       0.78      0.86      0.81        77
           1       0.89      0.82      0.85       107

    accuracy                           0.84       184
   macro avg       0.83      0.84      0.83       184
weighted avg       0.84      0.84      0.84       184

Uncertainity Estimates (Standard Deviation of Predictions): 
[0.07634792 0.22412033 0.06670716 0.10109269 0.09925788 0.1655916
 0.11357225 0.10209619 0.1089594  0.05327471 0.10475952 0.0947146
 0.12675999 0.12227052 0.09605157 0.10535139 0.10795429 0.11094782
 0.13018864 0.24997386 0.06727006 0.10422698 0.09683446 0.15952532
 0.06892495 0.09922195 0.10448691 0.10487525 0.12229149 0.10316883
 0.14900242 0.10935725 0.13148619 0.09949455 0.06188939 0.19782488
 0.11803515 0.08397848 0.12480014 0.13638012 0.08906145 0.13852166
 0.16531514 0.09789692 0.09442027 0.16001222 0.21956398 0.12701687
 0.06645422 0.07941779 0.10612574 0.0769038  0.0954558

Adding L2 Regularization, Batch Normalization, Early Stopping, and Learning Rate Schedule

In [41]:
class improvedModelMCDropout(Dropout):
    def call(self, inputs, training=None):
        return super().call(inputs, training=True)

improvedModel = Sequential()
kernel_reg_val = 0.006
dropout_val = 0.18
learning_rate_val = 0.0002
es_pat = 250
lr_pat = 125

improvedModel.add(Dense(128, activation='relu', kernel_regularizer=l2(kernel_reg_val)))
improvedModel.add(BatchNormalization())
improvedModel.add(improvedModelMCDropout(dropout_val))

improvedModel.add(Dense(64, activation='relu', kernel_regularizer=l2(kernel_reg_val)))
improvedModel.add(BatchNormalization())
improvedModel.add(improvedModelMCDropout(dropout_val))

improvedModel.add(Dense(32, activation='relu', kernel_regularizer=l2(kernel_reg_val)))
improvedModel.add(BatchNormalization())
improvedModel.add(improvedModelMCDropout(dropout_val))

improvedModel.add(Dense(1, activation='sigmoid'))

improvedModel.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_val), metrics=['accuracy'])

improvedModelCheckpoint = ModelCheckpoint('improvedNeuralNetworkModel.keras', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

earlyStopping = EarlyStopping(monitor='val_loss', patience=es_pat, restore_best_weights=True, verbose=1)
lrScheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=lr_pat, verbose=1, min_lr=1e-9)

improvedModel.fit(X_train, y_train, epochs=2000, batch_size=16, validation_split=0.1, callbacks=[improvedModelCheckpoint, earlyStopping, lrScheduler], verbose=1)

finalImprovedModel = tf.keras.models.load_model('improvedNeuralNetworkModel.keras', custom_objects={'improvedModelMCDropout' : improvedModelMCDropout})

def improvedMCPrediction(model, X, T=50):
    preds = np.array([model.predict(X) for _ in range(T)])
    return preds.mean(axis=0), preds.std(axis=0)

improvedMean, improvedStd = improvedMCPrediction(finalImprovedModel, X_test, T=50)

improvedModelThreshold = 0.5
improved_bin_pred = np.where(improvedMean > improvedModelThreshold, 1, 0)

print("Classification Report: ")
print(classification_report(y_test, improved_bin_pred))

Epoch 1/2000
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5512 - loss: 1.6507
Epoch 1: val_accuracy improved from -inf to 0.63514, saving model to improvedNeuralNetworkModel.keras
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.5516 - loss: 1.6503 - val_accuracy: 0.6351 - val_loss: 1.8819 - learning_rate: 2.0000e-04
Epoch 2/2000
 1/42 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5625 - loss: 1.6078
Epoch 2: val_accuracy did not improve from 0.63514
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6039 - loss: 1.5686 - val_accuracy: 0.5946 - val_loss: 1.6214 - learning_rate: 2.0000e-04
Epoch 3/2000
38/42 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6402 - loss: 1.5347 
Epoch 3: val_accuracy improved from 0.63514 to 0.70270, saving model to improvedNeuralNetworkModel.keras
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6405 - loss: 1.5324 - val_accuracy: 0.7027 - val_loss: 1.4351 - learning_rate: 2.0000e-04
Epoch 4/2000
41/42 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

In [42]:
org_accuracy = accuracy_score(y_test, binary_preds)
print(f"Original Model accuracy: {org_accuracy: .4f}")

mcDropout_accuracy = accuracy_score(y_test, mc_bin_pred)
print(f"MC Dropout Model accuracy: {mcDropout_accuracy: .4f}")

improved_model_accuracy = accuracy_score(y_test, improved_bin_pred)
print(f"Improved MC Dropout Model accuracy: {improved_model_accuracy: .4f}")

Original Model accuracy:  0.8098
MC Dropout Model accuracy:  0.8370
Improved MC Dropout Model accuracy:  0.8152
